In [64]:
import pandas as pd
import numpy as np
import os
import sys
import json
import requests
import gc
from tqdm import tqdm
from datetime import datetime, timedelta
import datetime as dt
import openpyxl
import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib as mpl

import statsmodels
import scipy.stats as ss
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler

from google.cloud import bigquery
from google.cloud.bigquery import job
import pandas as pd
import os
from tqdm import tqdm
import pandas as pd
PROJCECT = 'ballosodeuk'
bq = bigquery.Client(project=PROJCECT)

In [65]:
pd.get_option("display.max_columns")
pd.set_option("display.max_columns", 150)
pd.set_option("display.max_rows", 500)

In [66]:
query_name = "0608_행동데이터"
df = pd.read_parquet(f"../result/{query_name}.parquet")


- 유저아이디를 매우 많이 쓰는 유저가 블랙 유저일까? 추후 확인 필요. 지금은 매우 시간없음
- 일일로는 유저아이디 공백이 뚜렸했는데, 한달로 집계하니 유저아이디 공백이 없음. 개굳쓰

In [67]:
df['User_Id_cnt'] = df['User_Id'].apply(lambda x : len(x))


In [68]:
print("유저ID 2개 이상인 유저 : ", len(df[df['User_Id_cnt']>1]))
print(df['User_Id_cnt'].value_counts())


유저ID 2개 이상인 유저 :  1708
User_Id_cnt
1      590851
2        1516
3         133
4          19
5           9
7           3
6           3
10          2
16          2
8           2
9           2
43          1
89          1
11          1
17          1
18          1
12          1
77          1
15          1
47          1
53          1
61          1
20          1
101         1
71          1
110         1
19          1
14          1
Name: count, dtype: int64


In [69]:
df.drop(['adPopcorn_offerwall_reward','avati_offerwall_reward','rsp_reward','First_Cur_Cash','Last_Cur_Cash','Cum_Cash'],axis=1, inplace=True)
df.drop(['Joined_Date'],axis=1, inplace=True)
df.drop(['is_EasyLogin','is_CashLauncher','is_PushAgreed'],axis=1, inplace=True)


In [70]:
df.columns

Index(['Airbridge_Device_ID', 'User_Id', 'Platform', 'Sub_Division', 'Channel',
       'Campaign', 'tap_go__bf_list', 'bf_offerwall', 'coin_reward',
       'cp_reward_bf', 'chg_reward', 'chg_reward_cnt', 'chg_join',
       'cps_reward', 'bongtu_reward', 'invite_reward', 'cp_reward_homeCash',
       'spend', 'spend_cnt', 'avg_visit_interval', 'total_visits',
       'adClick_homeChg_ownedBanner', 'adClick_homeChg_adPopcorn',
       'adClick_homeChg_adPie', 'adClick_homeChg_mobion',
       'adClick_homeCash_coupang', 'adClick_homeCash_adPopcorn',
       'adClick_homeCash_adPie', 'adClick_home_adPopcorn',
       'adClick_homeBottom_owned', 'adClick_homeBottom_adPopcorn',
       'adClick_homeBottom_adPie', 'adClick_benefit_adPopcorn',
       'adClick_purchase_owned', 'adClick_chgList_adPopcorn',
       'adClick_chgList_adPie', 'adClick_chg_owned', 'adClick_chg_adPopcorn',
       'adClick_info_adPopcorn', 'adClick_exit_adPie',
       'adClick_benefit_bongtu_adPopcorn', 'adClick_benefit_butto

In [71]:
df.describe()

,tap_go__bf_list,bf_offerwall,coin_reward,cp_reward_bf,chg_reward,chg_reward_cnt,chg_join,cps_reward,bongtu_reward,invite_reward,cp_reward_homeCash,spend,spend_cnt,avg_visit_interval,total_visits,adClick_homeChg_ownedBanner,adClick_homeChg_adPopcorn,adClick_homeChg_adPie,adClick_homeChg_mobion,adClick_homeCash_coupang,adClick_homeCash_adPopcorn,adClick_homeCash_adPie,adClick_home_adPopcorn,adClick_homeBottom_owned,adClick_homeBottom_adPopcorn,adClick_homeBottom_adPie,adClick_benefit_adPopcorn,adClick_purchase_owned,adClick_chgList_adPopcorn,adClick_chgList_adPie,adClick_chg_owned,adClick_chg_adPopcorn,adClick_info_adPopcorn,adClick_exit_adPie,adClick_benefit_bongtu_adPopcorn,adClick_benefit_button_adPopcorn,adImp_homeChg_ownedBanner,adImp_homeChg_adPopcorn,adImp_homeChg_adPie,adImp_homeChg_mobion,adImp_homeCash_coupang,adImp_homeCash_adPopcorn,adImp_homeCash_adPie,adImp_home_adPopcorn,adImp_homeBottom_owned,adImp_homeBottom_adPopcorn,adImp_homeBottom_adPie,adImp_benefit_adPopcorn,adImp_purchase_owned,adImp_chgList_adPopcorn,adImp_chgList_adPie,adImp_chg_owned,adImp_chg_adPopcorn,adImp_info_adPopcorn,adImp_exit_adPie,adImp_benefit_bongtu_adPopcorn,adImp_benefit_button_adPopcorn,quiz_count,friend_get_link_count,friend_put_send_count,fund_receive_count,launcher,User_Id_cnt
count,592559.0,592559.0,592559.000000,592559.000000,5.925590e+05,592559.0,592559.0,592559.000000,5.925590e+05,5.925590e+05,592559.000000,5.925590e+05,592559.0,592559.000000,592559.0,592559.0,592559.0,592559.0,592559.0,592559.0,592559.0,592559.0,592559.0,592559.0,592559.0,592559.0,592559.0,592559.0,592559.0,592559.0,592559.0,592559.0,592559.0,592559.0,592559.0,592559.0,592559.0,592559.0,592559.0,592559.0,592559.0,592559.0,592559.0,592559.0,592559.0,592559.0,592559.0,592559.0,592559.0,592559.0,592559.0,592559.0,592559.0,592559.0,592559.0,592559.0,592559.0,592559.0,592559.0,592559.0,592559.0,592559.0,592559.000000
mean,9.033453,2.358673,252.039856,50.582136,1.045418e+03,7.813639,5.140727,21.164490,4.147084e+02,1.215547e+01,14.702165,9.131402e+02,0.12523,1.794917,15.275932,0.131082,0.060175,0.258692,1.182822,5.056977,0.205659,1.523847,0.055505,0.004631,0.017757,1.468915,0.019696,0.007083,0.003662,0.047783,0.000056,0.005716,0.000849,1.021196,0.007938,0.018672,0.131082,0.060175,0.258692,1.182822,5.056977,0.205659,1.523847,0.055505,0.004631,0.017757,1.468915,0.019696,0.007083,0.003662,0.047783,0.000056,0.005716,0.000849,1.021196,0.007938,0.018672,1.887532,0.183231,0.015551,0.013902,20.002383,1.004636
std,28.396747,27.238105,292.494629,97.695850,7.690626e+03,12.263893,6.632155,44.042483,7.368847e+03,1.349845e+03,20.640270,5.768628e+03,0.517002,2.499984,11.599359,0.740207,0.435594,2.191768,36.278668,8.855994,1.51067,6.817998,1.069592,0.230458,0.210864,22.920085,0.446083,0.117717,0.298398,0.576074,0.042869,0.22222,0.124485,16.72268,0.214063,0.177129,0.740207,0.435594,2.191768,36.278668,8.855994,1.51067,6.817998,1.069592,0.230458,0.210864,22.920085,0.446083,0.117717,0.298398,0.576074,0.042869,0.22222,0.124485,16.72268,0.214063,0.177129,17.308782,1.103192,0.187921,0.35853,71.200143,0.306517
min,0.0,0.0,0.000000,0.000000,0.000000e+00,0.0,0.0,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.0,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000
25%,0.0,0.0,0.000000,0.000000,0.000000e+00,0.0,0.0,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.0,1.000000,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000
50%,0.0,0.0,116.000000,0.000000,0.000000e+00,0.0,1.0,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.0,1.070000,15.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0

In [72]:
len(df)

592559

## 접속 지역 피쳐 
---
해외출시이력이 없으므로, 접속지역이 해외라면, VPN 접속 가능성이 크다. \
VPN 유저는 광고를 클릭하지 않을 가능성이 크다.

해외 유저는 약 1.4%

In [73]:
prov = df.groupby('Sub_Division').agg({'Airbridge_Device_ID':'count'}).sort_values(by='Airbridge_Device_ID', ascending=False)
prov_ct = prov.rank(method='dense')['Airbridge_Device_ID'].apply(lambda x : 1 if x > 55 else 0)
prov_rst = prov.merge(prov_ct,left_index=True, right_index=True, how='left')
prov_rst.columns = ['prv_id', 'prv_ct']
prov_rst.loc['Tokyo','prv_ct'] = 0 # 제주도 랭킹 낮으므로 수기 변환
prov_rst.loc['Ōsaka','prv_ct'] = 0 # 제주도 랭킹 낮으므로 수기 변환

prov_grp = prov_rst.groupby('prv_ct').agg({'prv_id':'sum'})

print("해외 접속유저 : ", np.round(prov_grp.values[0]/prov_grp.values[1],2))


해외 접속유저 :  [0.01]


In [74]:
# 국내외 지역의 카테고리화
prov_rank = prov_rst[['prv_id']].rank(ascending=False).rename(columns={'prv_id':'prv_rank'}) 
prov_rst = prov_rst.merge(prov_rank,how='left',left_index=True,right_index=True )
prov_rst['prv_rank'] = prov_rst.apply(lambda row: 0 if row['prv_ct'] == 0 else row['prv_rank'], axis=1)

prov_rst.reset_index(inplace=True)

In [75]:
norm_df_len = len(df)
print(norm_df_len) # 본 세트에 병합

df = df.merge(prov_rst, on='Sub_Division', how='left')

if norm_df_len == len(df):
    df.drop(['Sub_Division'],axis=1,inplace=True)
    print(len(df))
    print("병합 완료, 이상 없음")
    print( "Null 값 : ",f"{len(df.query("prv_ct.isna()"))}")

592559
592559
병합 완료, 이상 없음
Null 값 :  13280


In [76]:
print("지역 카테고리 지도")
prov_rst.query("prv_ct > 0")

지역 카테고리 지도


,Sub_Division,prv_id,prv_ct,prv_rank
0,Seoul,460932,1,1.0
1,Gyeonggi-do,24446,1,2.0
2,Ulsan,21829,1,3.0
3,Gyeongsangnam-do,9342,1,4.0
4,Busan,7477,1,5.0
5,Sejong-si,5793,1,6.0
6,Incheon,5719,1,7.0
7,Daegu,5405,1,8.0
8,Gyeongsangbuk-do,4699,1,9.0
9,Jeollabuk-do,4236,1,10.0


## Y값 통합
---

In [77]:
#Y값 위치 조회
loc_adClick_starts = df.columns.get_loc('adClick_homeChg_ownedBanner')
loc_adClick_ends = df.columns.get_loc('adClick_benefit_button_adPopcorn')
loc_adImp_starts = df.columns.get_loc('adImp_homeChg_ownedBanner')
loc_adImp_ends = df.columns.get_loc('adImp_benefit_bongtu_adPopcorn')

print(loc_adClick_starts)
print(loc_adClick_ends)
print(loc_adImp_starts)
print(loc_adImp_ends)

20
40
41
60


In [78]:
ad_cols = df.iloc[:,loc_adClick_starts:loc_adImp_ends].columns
adClick_cols = df.iloc[:,loc_adClick_starts:loc_adClick_ends].columns
adImp_cols = df.iloc[:,loc_adImp_starts:loc_adImp_ends].columns

adClick_df = pd.concat([df[['Airbridge_Device_ID']],df[adClick_cols]], axis=1) # 광고 df 생성 
adImp_df = pd.concat([df[['Airbridge_Device_ID']],df[adImp_cols]], axis=1) # 광고 df 생성 

df.drop(ad_cols,axis=1,inplace=True) # df 내 광고 열 전체 drop

In [79]:
ad_df.head(2)

,Airbridge_Device_ID,adClick_homeChg_ownedBanner,adClick_homeChg_adPopcorn,adClick_homeChg_adPie,adClick_homeChg_mobion,adClick_homeCash_coupang,adClick_homeCash_adPopcorn,adClick_homeCash_adPie,adClick_home_adPopcorn,adClick_homeBottom_owned,adClick_homeBottom_adPopcorn,adClick_homeBottom_adPie,adClick_benefit_adPopcorn,adClick_purchase_owned,adClick_chgList_adPopcorn,adClick_chgList_adPie,adClick_chg_owned,adClick_chg_adPopcorn,adClick_info_adPopcorn,adClick_exit_adPie,adClick_benefit_bongtu_adPopcorn,adClick_benefit_button_adPopcorn,adImp_homeChg_ownedBanner,adImp_homeChg_adPopcorn,adImp_homeChg_adPie,adImp_homeChg_mobion,adImp_homeCash_coupang,adImp_homeCash_adPopcorn,adImp_homeCash_adPie,adImp_home_adPopcorn,adImp_homeBottom_owned,adImp_homeBottom_adPopcorn,adImp_homeBottom_adPie,adImp_benefit_adPopcorn,adImp_purchase_owned,adImp_chgList_adPopcorn,adImp_chgList_adPie,adImp_chg_owned,adImp_chg_adPopcorn,adImp_info_adPopcorn,adImp_exit_adPie,adImp_benefit_bongtu_adPopcorn
0,9e588edb-6049-4610-99fd-2eb88c93cd96,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,9ab314ec-a3cc-463d-9469-432675367c6e,0,0,0,0,12,0,2,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12,0,2,0,0,0,3,0,0,0,0,0,0,0,0,0


### 추적 가능 매출 기여 매체로 통폐합
- 개재 위치 & 광고 매체 경우의 수는 20개 가량
- 많은 경우의 수 대비 데이터의 Sparse함은 너무 방대함.
- 따라서 아래의 룰로 광고를 묶는다.
    - 광고는 배너 CPC광고, CPS 광고로 나눈다.
    - 지면은 홈캐시와 홈캐시 외 지면으로 나눈다.
        - 홈캐시 지면 : 홈캐시-cps, 홈캐시-네트워크, 
        - 홈캐시외 지면 : non-홈캐시-cps, non-홈캐시-네트워크

In [80]:
ad_df.columns

Index(['Airbridge_Device_ID', 'adClick_homeChg_ownedBanner',
       'adClick_homeChg_adPopcorn', 'adClick_homeChg_adPie',
       'adClick_homeChg_mobion', 'adClick_homeCash_coupang',
       'adClick_homeCash_adPopcorn', 'adClick_homeCash_adPie',
       'adClick_home_adPopcorn', 'adClick_homeBottom_owned',
       'adClick_homeBottom_adPopcorn', 'adClick_homeBottom_adPie',
       'adClick_benefit_adPopcorn', 'adClick_purchase_owned',
       'adClick_chgList_adPopcorn', 'adClick_chgList_adPie',
       'adClick_chg_owned', 'adClick_chg_adPopcorn', 'adClick_info_adPopcorn',
       'adClick_exit_adPie', 'adClick_benefit_bongtu_adPopcorn',
       'adClick_benefit_button_adPopcorn', 'adImp_homeChg_ownedBanner',
       'adImp_homeChg_adPopcorn', 'adImp_homeChg_adPie',
       'adImp_homeChg_mobion', 'adImp_homeCash_coupang',
       'adImp_homeCash_adPopcorn', 'adImp_homeCash_adPie',
       'adImp_home_adPopcorn', 'adImp_homeBottom_owned',
       'adImp_homeBottom_adPopcorn', 'adImp_homeBottom_ad

In [90]:
# adClick_df.set_index('Airbridge_Device_ID',inplace=True)
ad_list = adClick_df.columns.tolist()
print("총 열 갯수 : ",len(ad_list))

home_cash = list(filter(lambda x : 'homeCash' in x, ad_list))
non_home_cash = list(filter(lambda x : 'homeCash' not in x, ad_list))

home_cash_cps = list(filter(lambda x : 'coupang' in x, home_cash))
home_cash_cpc = list(filter(lambda x : 'coupang' not in x, home_cash))

non_home_cash_cps = list(filter(lambda x : 'coupang' in x, non_home_cash))
non_home_cash_cpc = list(filter(lambda x : 'coupang' not in x, non_home_cash))

non_home_cash_cpc

총 열 갯수 :  20


['adClick_homeChg_ownedBanner',
 'adClick_homeChg_adPopcorn',
 'adClick_homeChg_adPie',
 'adClick_homeChg_mobion',
 'adClick_home_adPopcorn',
 'adClick_homeBottom_owned',
 'adClick_homeBottom_adPopcorn',
 'adClick_homeBottom_adPie',
 'adClick_benefit_adPopcorn',
 'adClick_purchase_owned',
 'adClick_chgList_adPopcorn',
 'adClick_chgList_adPie',
 'adClick_chg_owned',
 'adClick_chg_adPopcorn',
 'adClick_info_adPopcorn',
 'adClick_exit_adPie',
 'adClick_benefit_bongtu_adPopcorn']

## 외부 데이터셋 병합
---
1. 쿠팡 다이나믹
2. 쿠팡 혜택
3. 애드파이 (비율로 퉁)

## 피쳐 구간화
---

In [82]:
import warnings
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.preprocessing import KBinsDiscretizer, StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
warnings.filterwarnings(action='ignore')